## V-SDDP algorithm for linear-polyhedral stochastic optimal control problems under finite independant noises

Brief presentation of the example developped in [Akian, Chancelier, Tran, (in preparation)]. This note is meant for users somewhat familiar with stochastic optimal control problems. We use notations similar to [Carpentier, Chancelier, Cohen, De Lara, Stochastic Multi-stage Optimization (2015)].

### Linear-Polyhedral framework

We want to solve a risk-neutral discrete-in-time stochastic optimal control problem where for each time step $t \in \left\{0, \ldots T-1\right\}$ the costs are polyhedral:
$c_t(x,u,w) = \max_{i \in I_t} \langle c^{i,w}_t, (x;u) \rangle + d_t^{i,w} + \delta_{P_t^w}(x,u)$
where $I_t$ is a finite set and $P_t^w$ a polyhedron. 

The dynamic is linear: $f_t(x,u,w) = A^w_t x + B^w_t u$.

The final cost function $\psi$ is of the form $\psi(x) = \max_{i \in I_T} \langle c^{i}_T, x \rangle + \delta_{X_{T}}$ where ${X_{T}}$ is a polyhedron.

### Dynamic Programming
Assume that the stochastic process $(W_t)_{1\leq t \leq T-1}$ is discrete in time and made of independant r.v. with finite supports. In the hazard-decision framework we set $V_T = \psi$ and
$$ 
V_t(x) = B_t(V_{t+1})(x) := \mathbb{E} \left( \tilde{B}_t(V_{t+1})(x, W_t) \right),
$$
where for each realisation $w$ of the finite noise $W_t$:
$$
\tilde{B}_t(V_{t+1})(x,w) = \inf_u c_t(x,u,w) + V_{t+1}( f_t(x,u,w) ). 
$$
For every time step $t$ and for every realisation $w$ of the noise $W_t$ denote by $X_{t}(w)$ the state constraints, i.e. the projection of $P_t(w)$ on the states. Under recourse and regularity assumptions, one can show that
$$
\mathrm{dom}(V_t) = \cap_{w} X_t(w).
$$
We will denote by $X_t$ the domain of the value function $V_t$.

### Upper and lower approximations of the value functions

Upper approximations are of the form
$$
    \overline{V}_{t}^k = \inf_{1\leq j \leq k} g^j(\cdot) + \delta_{X_t}(\cdot).
$$

where $g^j = L_t \lVert \cdot - center_j \rVert_1 + value_at_center_j$ with $L_t$ being a Lipschitz constant for $V_t$ on $X_t$.

Lower approximations are of the form
$$
    \underline{V}_{t}^k = \sup_{1\leq j \leq k} \langle v_t^{j}, \cdot \rangle + \delta_{X_t}(\cdot).
$$


### The Problem-child trajectory (as coined by Regan Baucke)

A scenario $w$ is a realisation of the sequence of discrete random variable $(W_0, \ldots, W_{T-1})$.
Knowing some upper approximations $(\overline{V}_t)_t$ and lower approximations $(\underline{V}_t)_t$ of the value functions, compute the optimal trajectory for the lower approximations associated with the scenario $w^*$ which maximizes the future gap. Explicitly, it is the sequence of states $(x_0, \ldots, x_T)$ defined recursively by $x_0$ given and
$$
x_{t+1} = f_t(x_t, u_t(x_t, w^*_t), w^*_t),
$$
with $u_t(x_t, w) \in \mathrm{argmin}_u c_t(x,u,w) + \underline{V}_{t+1}(f_t(x, u, w))$ an optimal control at $x_t$ for the lower approximations when the realization of the noise is $w$. The worse realization of the noise $w^*_t$(the problem child) at the time step $t$ is defined by
$$
    w^*_t = \mathrm{argmax}_w (\overline{V}_{t+1} - \underline{V}_{t+1})(f_t(x_t, u_t(x_t, w), w)).
$$

### V-SDDP algorithm
Assume constructed upper approximations $(\overline{V}_t^k)_t$ and lower approximations $(\underline{V}_t^k)_t$ of the value functions. At iteration $k+1$, the upper approximations are refined by adding one more quadratic to the current collection. It is computed as follows:

1) If the gap $(\overline{V}_0^k - \underline{V}_0^k) (x_0) < \epsilon$, then stop. Otherwise, compute the Problem-child trajectory $(x_0, \ldots, x_T)$ for the current approximations. Refine backward in time both approximations along the Problem-child trajectory.

2) Time step t = T. Compute a subgradient $\lambda$ of $V_T$ at $x_T$. Add the associated cut $\phi$ to the lower approximations, $\underline{V}_{T}^{k+1} = \sup( \underline{V}_T^k, \phi)$.

Compute a $c$-quadratic function $q$ equal to $V_T$ at $x_T$ with gradient at $x_T$ equal to $\lambda$. Set $\overline{V}_{T}^{k+1} = \inf( \overline{V}_T^k, q)$

3) Time step t < T. 

3.1) Lower approximations improvement by LP. Solve for each realization of the noise $w$, one LP, $\tilde{B}_t(\underline{V}_{t+1}^k)(x_t, w)$ with the state $x$ as variable but constrained to have $x = x_t$. Compute an optimal multiplier $\lambda$ associated with this constraint. This yields a new cut $x \mapsto \phi(x,w) = \langle \lambda, x \rangle + \text{value-at-origin}$ where
$$
\text{value-at-origin} = \tilde{B}_t(\underline{V}_{t+1}^k)(x_t, w) - \langle \lambda, x_t \rangle.
$$
Set $\underline{V}_{t}^{k+1} = \sup \left( \underline{V}_t^k, \mathbb{E}\left( \phi(\cdot, W_t \right)\right)$.

3.2) Upper approximations improvement by solving $k*\lvert W \rvert$ LPs.

Iterate until $t = 0$. Then go back to step 1).

In [1]:
using JuMP, Gurobi, LinearAlgebra, MathOptInterface

This Notebook additionally needs a working licence of Gurobi. A free academic licence can be found on the official website of Gurobi.

### TO DO : Precomputing $X_t$. 
https://www.sciencedirect.com/science/article/pii/0304397578900518 ?

TO DO : Lipschitz constant of the Vt assmued to be known.

### TO DO : Add computation of admissible sequence of states (it does not need to be a trajectory) instead of assuming that (x0, x0, ..., x0) is admissible

TO DO (Lesser): Add a V function for each omega (in temp). Truly add the mean of those V functions, it is polyhedral. Compute B_t of this polyhedral function at x_t with omega given, add a V function and so on.

Hope that the mean of V will be smoother and more suited to the mean operator.

In [2]:
function main_TDP(itermax, epsilon, T, Xi, x0, dyn, cost, coupling, state_cons, Lipschitz)
    global GUROBI_ENV = Gurobi.Env()
    gap_at0 = Inf;
#    global c = curvature; 
    
    iter = 1; # We will do the first iteration "manually"
    # Initialization
    F_up = Array{Any}(undef, T);
    F_down = Array{Any}(undef, T);
    PC_traj = Array{Any}(undef, T); 
    best_u_down = Array{Any}(undef, T-1);
    best_u_up = Array{Any}(undef, T-1);
    val_up = zeros(Float64, T);
    val_down = zeros(Float64, T);
    trajectories = Array{Any}(undef, T);
    n = length(x0); # Dimension of the states
    m = length(dyn[1][1][2][1,:]); # Dimension of the controls
    for t in 1:T
        F_up[t] = [ [zeros(Float64, n), +Inf] ];
        F_down[t] = [ [zeros(Float64, n), -Inf] ];
        PC_traj[t] = x0; # Assuming that x_0 is admissible for all time
        trajectories[t] = [x0];
        if t < T
            W = length(Xi[t]);
            best_u_down[t] = Array{Any}(undef, W);
            best_u_up[t] = Array{Any}(undef, W);
            for w in 1:W
                best_u_down[t][w] = zeros(Float64, m); # Assuming (x0, 0, w) admissible
                best_u_up[t][w] = zeros(Float64, m);
            end
        end
        val_up[t] = +Inf;
        val_down[t] = -Inf;
    end
    println("Initialisation finie")
    # Iteration step
   # while (gap_at0 > epsilon) && (iter < itermax)
    while (iter < itermax)
    println("---------")
        println("Iteration numero : ", iter)
        # Backward phase: improve the current approximations, stocks best_u and val
        println("Problem Child traj : ", PC_traj)
        Iteree_Up = Selection_V(PC_traj, F_up, dyn, cost, coupling, state_cons, Xi, T, Lipschitz); 
        Iteree_Down = Selection_SDDP(PC_traj, F_down, dyn, cost, coupling, state_cons, Xi);
        if (Iteree_Up[1][T] != [zeros(n,1), -Inf]) && ((Iteree_Up[1][T] in F_up[T]) == false)
            F_up[T] = vcat(F_up[T], [Iteree_Up[1][T]]);
        end
        if (Iteree_Down[1][T] != [zeros(n,1), +Inf])  && ((Iteree_Down[1][T] in F_down[T]) == false) 
            F_down[T] = vcat(F_down[T], [Iteree_Down[1][T]]);
        end       
        for t in 1:(T-1)
                best_u_up[t] = Iteree_Up[2][t];
                best_u_down[t] = Iteree_Down[2][t];
            if  (Iteree_Up[1][t] != [zeros(n,1), -Inf,0]) && ((Iteree_Up[1][t] in F_up[t]) == false)
                F_up[t] = vcat(F_up[t], [Iteree_Up[1][t]]);
            end
            if (Iteree_Down[1][t] != [zeros(n,1), +Inf])  && ((Iteree_Down[1][t] in F_down[t]) == false) 
                F_down[t] = vcat(F_down[t], [Iteree_Down[1][t]]);
            end
        end
        # Stopping criterion update
        gap_at0 = Iteree_Up[3][1] - Iteree_Down[3][1];
        iter += 1;
        # Prints some information about the current approximations
        for t in 1:T
            println("Gap at t = ", t-1, ": ", Iteree_Up[3][t] - Iteree_Down[3][t])
            println("   Length F_up[t] = ", length(F_up[t]))
            println("   Length F_down[t] = ", length(F_down[t]))
        end
        # Generates the Problem-Child trajectory
        PC_traj = Problem_Child_trajectory(F_up, F_down, x0, best_u_down, dyn, T, Lipschitz);
        for t in 1:T
            trajectories[t] = vcat(trajectories[t], [PC_traj[t]]);
        end
    end
    return(F_up, F_down, best_u_up, best_u_down, trajectories)
end

main_TDP (generic function with 1 method)

### Problem-child trajectory and upper/lower approximations evaluations

In [3]:
function Problem_Child_trajectory(F_up, F_down, x0, best_u, dyn, T, Lipschitz)
    traj = Array{Any}(undef, T);
    traj[1] = x0;
    for t in 1:(T-1)
        println("--")
        println("Time t = ",t -1)
        currentGap = -Inf;
        futurState_W = -Inf*ones(length(x0), 1)
        for w in 1:length(Xi[t])
            futurState_W = dyn[t][w][1]*traj[t] + dyn[t][w][2]*best_u[t][w]
            newGap = Eval_V(F_up[t+1], futurState_W, Lipschitz[t+1]) - Eval_SDDP(F_down[t+1], futurState_W);
            newGap = max(newGap, 0)
            if newGap >= currentGap
                traj[t+1] = futurState_W;
                currentGap = newGap;
            end
        end
        println("currentGap = ", currentGap, " futurState_W = ", futurState_W)
    end
    return(traj)
end

Problem_Child_trajectory (generic function with 1 method)

In [4]:
function Eval_V(F_up, state, L)
    current_eval = Inf;
    for j in 2:length(F_up)
        new_eval = L*sum(abs.(state - F_up[j][1])) + F_up[j][2];
        if new_eval < current_eval
            current_eval = new_eval;
        end
    end
    return(current_eval)
end

Eval_V (generic function with 1 method)

In [5]:
function Eval_SDDP(F_down, state)
    current_eval = -Inf;
    for j in 2:length(F_down)
        cut_eval = (F_down[j][1]'*state)[1] + F_down[j][2];
        if cut_eval > current_eval
            current_eval = cut_eval;
        end
    end
    return(current_eval)
end

Eval_SDDP (generic function with 1 method)

### SDDP selection function

In [6]:
function Selection_SDDP(x, F_down,  dyn, cost, coupling, state_cons, Xi)
    new_down = Array{Any}(undef, T);
    best_u = Array{Any}(undef, T-1);
    traj_values = -Inf*ones(Float64, T);
    # The final time step T is a bit particular
    output = Selection_SDDP_Finalstep(x[T], cost[T], state_cons[T]);
    new_down[T] = output[1];
    traj_values[T] = output[2];
    for t in (T-1):-1:1
        # global new_down, best_u, values
        W = length(Xi[t]); # Number of elements in the support of the noise
        tmp_new_originvalue = zeros(Float64, W);
        tmp_new_slope = zeros(Float64, length(x[1]), W);
        best_u[t] = Array{Any}(undef, W);
        tmp_traj_values = zeros(Float64, W);
        tmp_new_down = vcat(F_down[t+1], [new_down[t+1]]);
        # Stocks for each noise, the new function and best control
        for w in 1:W
            output = Selection_SDDP_Onestep(x[t], tmp_new_down, dyn[t][w], cost[t][w],
            coupling[t][w], state_cons[t+1], Xi[t][w]);
            tmp_new_slope[:, w] = output[1][1];
            tmp_new_originvalue[w] = output[1][2];
            best_u[t][w] = output[2];
        end
        # We add the mean cut 
        new_down[t] = [sum(Xi[t]'.*tmp_new_slope, dims=2), Xi[t]'*tmp_new_originvalue];
        traj_values[t] = (new_down[t][1]'*x[t])[1] + new_down[t][2];
    end
    return(new_down, best_u, traj_values)
end

Selection_SDDP (generic function with 1 method)

In [7]:
function Selection_SDDP_Finalstep(xT, cost, XT)
    M = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(M, "OutputFlag", 0)
    n = length(xT);
    K = length(cost);
    @variable(M, x[1:n])
    @variable(M, mu)
    @constraint(M, on_mu[k in 1:K], mu >= (cost[k][1]'*x)[1] + cost[k][2])
    @constraint(M, state_constraints, XT[2] .>= XT[1]*x)
    @constraint(M, fixing_x, x .== xT)
    @objective(M, Min, mu)
    optimize!(M)
    if termination_status(M) == MathOptInterface.INFEASIBLE
        println("Infeasible Problem")
        value_objective = +Inf;
        value_at_origin = -Inf;
        lambda = zeros(n, 1);
    else
        value_objective = objective_value(M);
        lambda = dual.(fixing_x);
        value_at_origin = value_objective - (lambda'*xT)[1];
    end
    return([lambda, value_at_origin], value_objective)
end

Selection_SDDP_Finalstep (generic function with 1 method)

In [8]:
function Selection_SDDP_Onestep(xt, F_down, dyn, cost, coupling, Xtplus1, Xi)
    M = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(M, "OutputFlag", 0)
    n = length(xt); # Dimension of the states
    m = length(dyn[2][1,:]); # Dimension of the controls
    J = length(F_down); # Number of cuts defining the current approx. of V_{t+1}
    K = length(cost); # Number of cuts defining the cost function
    @variable(M, x[1:n])
    @variable(M, u[1:m])
    @variable(M, mu)
    @variable(M, gamma)
    var = vcat(x[1:n], u[1:m]);
    futurstate = dyn[1]*x[1:n] + dyn[2]*u[1:m];
    lambda = zeros(Float64, m, 1);
    @constraint(M, fixing_x, x .== xt)
    @constraint(M, on_mu[k in 1:K], mu >= (cost[k][1]'*var)[1] + cost[k][2])
    @constraint(M, on_gamma[j in 2:J], gamma >= (F_down[j][1]'*futurstate)[1] + F_down[j][2])
    @constraint(M, coupling_constraints, coupling[2] .>= coupling[1]*var)
    @constraint(M, futurstate_constraints, Xtplus1[2] .>= Xtplus1[1]*futurstate)
    @objective(M, Min, mu + gamma)
    optimize!(M)
    if termination_status(M) == MathOptInterface.INFEASIBLE
        println("Infeasible Problem")
        return([zeros(n,1), -Inf], zeros(m,1),+Inf)
    else
        lambda = dual.(fixing_x);
    end
    return([lambda, objective_value(M) - (lambda'*xt)[1]], value.(u))
end

Selection_SDDP_Onestep (generic function with 1 method)

### V selection function

In [9]:
function Selection_V(traj, F_up, dyn, cost, coupling, state_cons, Xi, T, Lipschitz) 
    new_up = Array{Any}(undef, T);
    best_u = Array{Any}(undef, T-1);
    values = Inf*ones(Float64,T);
    k = length(F_up);
    # The final time step T is particular
    output = Selection_V_Finalstep(traj[T], cost[T], state_cons[T]);
    new_up[T] = output[1];
    values[T] = output[2];
    for t in (T-1):-1:1
        W = length(Xi[t]);
        best_u[t] = Array{Any}(undef, W);
        tmp_center = zeros(Float64, length(traj[t]), W);
        tmp_centervalue = zeros(Float64, W);
        # Stocks for each noise, the new function and best control
        tmp_newup_nextstep = vcat(F_up[t+1], [new_up[t+1]]);
        for w in 1:W
            output = Selection_V_Onestep(traj[t], tmp_newup_nextstep, dyn[t][w], 
                cost[t][w], coupling[t][w], state_cons[t+1], Lipschitz[t+1]);
            tmp_centervalue[w] = output[1]; # = B_t(q_i)(x_t, w)
            best_u[t][w] = output[2];
         end
        # We add the a V-function (hence valid) equal to E[B_t(q_{i(W)})(x, W)] at x_t (hence tight)
        centerval = sum(Xi[t].*tmp_centervalue)# = E(q(a, W_t))
        new_up[t] = [traj[t], centerval];
        values[t] = centerval;
    end
    return(new_up, best_u, values)
end

Selection_V (generic function with 1 method)

In [10]:
function Selection_V_Finalstep(xT, cost, XT)
    M = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(M, "OutputFlag", 0)
    n = length(xT);
    K = length(cost);
    @variable(M, mu)
    @constraint(M, on_mu[k in 1:K], mu >= (cost[k][1]'*xT)[1] + cost[k][2])
    @constraint(M, state_constraints, XT[2] .>= XT[1]*xT)
    @objective(M, Min, mu)
    optimize!(M)
    if termination_status(M) == MathOptInterface.INFEASIBLE
        println("Infeasible Problem")
        return([zeros(n,1), +Inf, 0], +Inf )
    else
        value_objective = objective_value(M);
    end
    return([xT, value_objective], value_objective)
end

Selection_V_Finalstep (generic function with 1 method)

In [11]:
function Selection_V_Onestep(xt, F_up, dyn, cost, coupling, Xtplus1, L)
    value_at_xt = +Inf;
    uopt = zeros(Float64, m, 1);
    J = length(F_up)
    for j in 2:J
        tmp = Selection_V_Onestep_slave(xt, F_up[j], dyn, cost, coupling, Xtplus1, L);
        if tmp[1] < value_at_xt
            value_at_xt = tmp[1];
            uopt = tmp[2];
        end
    end
    return(value_at_xt, uopt)
end

Selection_V_Onestep (generic function with 1 method)

In [12]:
function Selection_V_Onestep_slave(xt, F_up, dyn, cost, coupling, Xtplus1, L)
    M = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(M, "OutputFlag", 0)
    m = length(dyn[2][1,:]); # Dimension of the controls
    n = length(xt);
    K = length(cost); # Number of cuts defining the cost function
    uopt = zeros(Float64, m, 1);
    @variable(M, u[1:m])
    var = vcat(xt, u[1:m]);
    futurstate = dyn[1]*xt + dyn[2]*u[1:m];
    @variable(M, mu)
    @variable(M, gamma[1:n])
    @constraint(M, on_mu[k in 1:K], mu >= (cost[k][1]'*var)[1] + cost[k][2])
    @constraint(M, on_gamma_pos[i in 1:n], gamma[i] >= L*(futurstate[i] - F_up[1][i]) + F_up[2])
    @constraint(M, on_gamma_neg[i in 1:n], gamma[i] >= L*(F_up[1][i] - futurstate[i]) + F_up[2])
    @constraint(M, coupling_constraints, coupling[2] .>= coupling[1]*var)
    @constraint(M, futurstate_constraints, Xtplus1[2] .>= Xtplus1[1]*futurstate)
    @objective(M, Min, mu + sum(gamma))
    optimize!(M)
    if termination_status(M) == MathOptInterface.INFEASIBLE  
        println("Infeasible Problem")
        return([zeros(n,1), +Inf, 0], zeros(m,1), +Inf)
    else
    return(objective_value(M), value.(u))  
    end
end

Selection_V_Onestep_slave (generic function with 1 method)

## Toy example

In [13]:
using Random
Random.seed!(134) ## Fixe l'aléa à chaque run
#Random.seed!(5678797)
#### 
n = 1; m = 1; 
itermax = 100; epsilon = 0.001; T = 3; control_bound = 3; state_bound = 10; Lipschitz = 1.0*ones(T,1);
Lipschitz[2] = 1 + 2*Lipschitz[3]*(n+1);
Lipschitz[1] = 1 + 4*Lipschitz[2]*(n+1);
#x0 = ones(n,1) + rand(n,1);
x0 = -0*ones(n,1);
Xi = [[0.2, 0.2, 0.6], [0.3, 0.3, 0.2, 0.2]];
dyn = Array{Any}(missing, T-1);
cost = Array{Any}(missing, T);
cost[T] = [ [ones(n, 1), -1], [zeros(n, 1), 1], [-ones(n, 1), -1] ]
coupling = Array{Any}(missing, T-1);
state_cons = Array{Any}(undef, T);
state_cons[T] = [vcat(Matrix{Float64}(I, n, n), -Matrix{Float64}(I, n, n)), state_bound*ones(2*n)];
for t in 1:(T-1)
    W = length(Xi[t]);
    dyn[t] = Array{Any}(missing, W);
    cost[t] = Array{Any}(missing, W);
    coupling[t] = Array{Any}(missing, W);
    state_cons[t] = [vcat(Matrix{Float64}(I, n, n), -Matrix{Float64}(I, n, n)), state_bound*ones(2*n)]
    for w in 1:W
    coupling[t][w] = [vcat(Matrix{Float64}(I,n+m,n+m),-Matrix{Float64}(I,n+m,n+m)), 
            repeat(vcat(state_bound*ones(n), control_bound*ones(m)),2,1)];
        if w == 1
            dyn[t][w] = [Matrix{Float64}(I,n,n) + rand(n,n), (1/2)*Matrix{Float64}(I,m,n) + rand(m,n)];
            cost[t][w] = [[ones(n+m, 1), -1]];
        elseif w == 2
            dyn[t][w] = [Matrix{Float64}(I,n,n) - rand(n,n), (1/2)*Matrix{Float64}(I,m,n) + rand(m,n)];
            cost[t][w] = [[zeros(n+m, 1), 1]];
        elseif w == 3
            dyn[t][w] = [Matrix{Float64}(I,n,n), (1/2)*Matrix{Float64}(I,m,n) + rand(m,n)];
            cost[t][w] = [[ones(n+m, 1), -1]]; 
        else
            dyn[t][w] = [Matrix{Float64}(I,n,n), (1/2)*Matrix{Float64}(I,m,n) + rand(m,n)];
            cost[t][w] = [[zeros(n+m, 1), 3]];
        end
    end
end
result = main_TDP(itermax, epsilon, T, Xi, x0, dyn, cost, coupling, state_cons, Lipschitz);

Academic license - for non-commercial use only
Initialisation finie
---------
Iteration numero : 1
Problem Child traj : Any[[0.0], [0.0], [0.0]]
Gap at t = 0: 4.984475355499453
   Length F_up[t] = 2
   Length F_down[t] = 2
Gap at t = 1: 1.407647460268338
   Length F_up[t] = 2
   Length F_down[t] = 2
Gap at t = 2: 0.0
   Length F_up[t] = 2
   Length F_down[t] = 2
--
Time t = 0
currentGap = 16.240092313218256 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 6.648997920060241 futurState_W = [-5.049122282490253]
---------
Iteration numero : 2
Problem Child traj : Any[[0.0], [-2.6968081550818033], [-6.648997920060241]]
Gap at t = 0: 2.7406411006545204
   Length F_up[t] = 3
   Length F_down[t] = 3
Gap at t = 1: 1.0678121334537454
   Length F_up[t] = 3
   Length F_down[t] = 3
Gap at t = 2: 0.0
   Length F_up[t] = 3
   Length F_down[t] = 3
--
Time t = 0
currentGap = 4.049196995989024 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 2.0 futurState_W = [0.2554900811

Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 19
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 20
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1:

Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 37
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 38
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1:

Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 56
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 57
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1:

Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 75
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 76
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1:

Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 94
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1: -1.1102230246251565e-16
   Length F_up[t] = 11
   Length F_down[t] = 5
Gap at t = 2: 0.0
   Length F_up[t] = 12
   Length F_down[t] = 3
--
Time t = 0
currentGap = 0.0 futurState_W = [-2.3248822503109645]
--
Time t = 1
currentGap = 0.0 futurState_W = [0.02743187709748529]
---------
Iteration numero : 95
Problem Child traj : Any[[0.0], [-2.3248822503109645], [0.02743187709748529]]
Gap at t = 0: 0.0
   Length F_up[t] = 11
   Length F_down[t] = 3
Gap at t = 1:

## (Classical) Dynamic Programming

In [14]:
"""
    Is only suitable for the previous example: for instance, it assumes that the 
    coupling constraints are a box, and the same one for every time step and noise.
"""
function DynamicProgramming(T, Xi, x0, dyn, cost, coupling, state_cons, control_bound)
    controls = -(control_bound):0.3:control_bound;
    # Forward : on calcule tous les états admissibles atteignables
    state = Array{Any}(undef, T) # Contient tous les états possibles
    value = Array{Any}(undef, T) # Contient tous les V_t(x) pour tout t et pour tout x
    state[1] = x0;
    state[1] = vcat(state[1], x0-[2.0])
    state[1] = vcat(state[1], x0+[2.0])
    value[1] = [];
    for t in 2:T
        state[t] = [];
        value[t] = [];
        for current_state in state[t-1]
            for w in 1:length(Xi[t-1])
                for u in controls
                    futur_state = dyn[t-1][w][1]*current_state + dyn[t-1][w][2]*u;
                    if  state_cons[t][2] >= vec(state_cons[t][1]*futur_state)
                        state[t] = vcat(state[t], futur_state); # Admissible state at t from a state at t-1
                    end
                end
            end
        end
    end
    # On supprime les états doublons
    for t in 2:T
       state[t] = unique(state[t]); 
    end
    # Backward : on calcule en chaque état la fonction valeur en commençant par t=T
    for current_state in state[T]
        value[T] = vcat(value[T], Slave_FinalStep(current_state, cost[T]))
    end
    for t in (T-1):-1:1
        for current_state in state[t]
            L = length(Xi[t]); # 
            tmp = zeros(Float64, L); # Array containing each, B_t(V_t+1)(current_state, w)
            for w in 1:L
                tmp2 = Inf # Values of the reachable states from (current_state, w)
                for u in controls
                    futur_state = dyn[t][w][1]*current_state + dyn[t][w][2]*u;
                    if state_cons[t+1][2] >= vec(state_cons[t+1][1]*futur_state)
                        index = findall(x-> x == futur_state[1], state[t+1]); # Dependent on dim = 1
                        tmp2 = min(tmp2, Slave_OneStep(current_state, u, cost[t][w], value[t+1][index]));
                    end
                end
                tmp[w] = tmp2; # Best value among all reachable states from (current_state, w)
            end
            value[t] = vcat(value[t], (Xi[t]'*tmp)[1]) # Contains every E[B_t(V_t+1)(current_state, W_t)]
        end
    end
    return(state, value)
end

DynamicProgramming

In [15]:
function Slave_FinalStep(xT, cost)
    M = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(M, "OutputFlag", 0)
    K = length(cost);
    @variable(M, mu)
    @constraint(M, on_mu[k in 1:K], mu .>= (cost[k][1]'*xT)[1] + cost[k][2])
    @objective(M, Min, mu)
    optimize!(M)
    if termination_status(M) == MathOptInterface.INFEASIBLE
        println("Infeasible Problem")
        value_objective = +Inf;
    else
        value_objective = objective_value(M);
    end
    return(value_objective)
end

Slave_FinalStep (generic function with 1 method)

In [16]:
function Slave_OneStep(xt, u, cost, Vtplus1) 
    M = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(M, "OutputFlag", 0)
    K = length(cost);
    @variable(M, mu)
    @constraint(M, on_mu[k in 1:K], mu .>= (cost[k][1]'*[xt;u])[1] + cost[k][2])
    @objective(M, Min, mu)
    optimize!(M)
    if termination_status(M) == MathOptInterface.INFEASIBLE
        println("Infeasible Problem")
        value_objective = +Inf;
    else
        value_objective = objective_value(M) + Vtplus1[1];
    end
    return(value_objective)
end

Slave_OneStep (generic function with 1 method)

In [17]:
DP_result = DynamicProgramming(T, Xi, x0, dyn, cost, coupling, state_cons, control_bound);

### Plots of $\underline{V}_t^k$ and $\overline{V}_t^k$

In [18]:
using Plots

In [64]:
function plotting1D(Y_up, Y_down, traj, abscisses, t, abs_dynpro, ord_dynpro, L)
    # Initialisation
    absi = sort(vec(abs_dynpro[t]));
    MinY_up = +Inf*ones(length(abscisses));
    MaxY_down = -Inf*ones(length(abscisses));
    perm = sortperm(vec(abs_dynpro[t]));
    ord = vec(ord_dynpro[t][perm]);
    yb = (-5,10);
    # Iteratively plots the new functions to be added along with the current approximations
    for i in 2:length(Y_up)
        Y_up_i = L*abs.(abscisses .- Y_up[i][1][1]) .+ Y_up[i][2];
        MinY_up = min.(MinY_up, Y_up_i);
        time = string("t = ", t)
        iteration = string("Iteration = ", i)
        plot(abscisses, MinY_up,color=[:red],w=3, ylims=yb,label =  
                string("Current approx up: ", string(i-1), " at t = ", string(t)), fmt = :pdf,
                xlabel = time, ylabel = iteration)
        plot!(abscisses, Y_up_i, ylims=yb,color=[:orange], label = "New up", fmt = :pdf)
        if i <= length(Y_down)
            Y_down_i = Y_down[i][1][1,1]*abscisses .+ Y_down[i][2];
            MaxY_down = max.(MaxY_down, Y_down_i);
            plot!(abscisses, Y_down_i, ylims=yb, color=[:green], label = "New down", fmt = :pdf)
        end
        plot!(abscisses, MaxY_down,color=[:blue],w=3, ylims=yb, label = "Current approx down", fmt = :pdf)
        p=plot!(absi, ord, ylims=yb, w=1, color=[:black], label = "Valeur par dyn pro", fmt = :pdf)
        vline!(traj[i-1], label = "x_t^(i-1)", linestyle = :dash);
        name = string("V_SDDP_",t,"_",i, ".pdf")
        savefig(name)
    end
end

plotting1D (generic function with 1 method)

In [62]:
t = 3;
plotting1D(result[1][t], result[2][t], result[5][t], -7.5:0.1:7.5, t, DP_result[1], DP_result[2], Lipschitz[t])

bonjour
bonsoir
bonjour
bonsoir
bonjour
bonjour
bonjour
bonjour
bonjour
bonjour
bonjour
bonjour
bonjour


In [63]:
t = 2;
plotting1D(result[1][t], result[2][t], result[5][t], -7.5:0.1:7.5, t, DP_result[1], DP_result[2], Lipschitz[t])

bonjour
bonsoir
bonjour
bonsoir
bonjour
bonsoir
bonjour
bonsoir
bonjour
bonjour
bonjour
bonjour
bonjour
bonjour


In [65]:
t = 1;
plotting1D(result[1][t], result[2][t], result[5][t], -7.5:0.1:7.5, t, DP_result[1], DP_result[2], Lipschitz[t])